In [23]:
import urllib.request
from bs4 import BeautifulSoup
import os
from joblib import Parallel, delayed
from multiprocessing import Pool
from multiprocessing import Process
import multiprocessing
import time
import requests 
import urllib.request
import json
import csv  
import tqdm
import PyPDF2 

from multiprocessing import Pool, freeze_support, RLock
from multiprocessing import current_process
from p_tqdm import p_map, p_umap, p_imap, p_uimap
from clint.textui import progress

In [24]:
def get_text_html(link):
    time.sleep(1)
    return BeautifulSoup(requests.get(url=link).text, 'html.parser')

In [25]:
def create_dir(path):
    if os.path.exists(path) == False:
        os.mkdir(path)

In [26]:
def get_pages_by_author(data):
    pages = {}
    name_author = ''
    for item in data:
        tag_name = item.name
        if tag_name == 'h2':
            name_author = item.text.replace(' ¶','')
#             print(name_author)
        elif tag_name == 'ul':
            name_book = item.text
            links_book = []
            for i in item.findChildren('li', recurdive=False):
                if 'English' in i.text:
                    links_on_book = i.find_all('a')
                    for link in links_on_book:
                        if 'ebooks' in link['href']:
                            links_book.append({f"https://www.gutenberg.org{link['href']}": link.text})
            if len(links_book) > 0:
                pages[name_author] = links_book
    return pages

In [27]:
def get_direct_reference(link):
    result = 'empty'
    page = get_text_html(link)
    data = page.find_all("td", {"class": "noscreen"})
    for i in data:
        inner_text = i.text
        if '.txt' in inner_text:
            result = inner_text
        elif '.pdf' in inner_text:
            result = inner_text
    return result

In [28]:
def download_by_link(link, path):
    if '.txt' in link:
        urllib.request.urlretrieve(link, path)
    elif '.pdf' in link:
        print('Working with pdf: ', link)
        temp_path = f"{path} temp.pdf"
        urllib.request.urlretrieve(link, temp_path)
        
        pdffileobj=open(temp_path,'rb')
        pdfReader = PyPDF2.PdfFileReader(pdffileobj)
        count_pages = pdfReader.numPages
        file1=open(f"{path}","a")
        for i in range(count_pages):
            file1.write(pdfReader.getPage(i).extractText())
        file1.close()
        os.remove(temp_path)

In [29]:
def download_by_item(item):
    path = item[0]
    name_author = item[1][0]
    create_dir(f"{path}/{name_author}")
    list_books = item[1][1]
    for page_info in list_books:
        for link in page_info.keys():
            direct_reference = get_direct_reference(link)
            download_by_link(direct_reference, f"{path}/{name_author}/{page_info[link][:250]}.txt")

In [30]:
def get_pairs_link_path(dict_info, dir_name):
    count = 1
    pairs_dir_name_book_info = []
    dict_info_items = list(dict_info.items())
    
    for item in dict_info_items:
#         if count > 1:
#             break
        pairs_dir_name_book_info.append((dir_name, item))
#        print(count)
        count += 1
    return pairs_dir_name_book_info

In [31]:
def download_files(link_path):
    download_by_link(link_path[0], link_path[1])

In [32]:
url = "https://www.gutenberg.org/browse/authors/a"
page = get_text_html(url)

# pages = get_pages_by_author(page.find("div", {"class": "pgdbbyauthor"}).find('div').children)
pages = get_pages_by_author(page.find("div", {"class": "pgdbbyauthor"}).children)

In [ ]:
dir_name = f"../{'A'}"
create_dir(dir_name)
pairs_link_path = get_pairs_link_path(pages, dir_name)
print('Pairs was made: ', len(pairs_link_path))
# for i in pairs_link_path:
#     print('Was download: ', i)
#     download_by_item(i)

pool = Pool()
# result = pool.apply_async(download_by_item, pairs_link_path)
# print(result.get(timeout=1))

# if __name__ == '__main__':
#     processlist = []
#     for pair in pairs_link_path:
#         processlist.append(Process(target=download_by_item, args=(pair,)))
    
#     for t in processlist:
#         t.start()
    
#     for t in processlist:
#         t.join()

# pool.imap_unordered(download_by_item, pairs_link_path)
for _ in tqdm.tqdm(pool.imap(download_by_item, pairs_link_path), total=len(pairs_link_path)):
    pass
print('Done')

Pairs was made:  924


  1%|▏                                        | 5/924 [02:22<7:17:30, 28.56s/it]
Process ForkPoolWorker-42:
Process ForkPoolWorker-39:
Process ForkPoolWorker-40:
Process ForkPoolWorker-38:
Process ForkPoolWorker-37:
Process ForkPoolWorker-36:
Process ForkPoolWorker-41:
Traceback (most recent call last):
Process ForkPoolWorker-35:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/

  File "/usr/lib/python3/dist-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "/usr/lib/python3/dist-packages/requests/sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "/usr/lib/python3.8/urllib/request.py", line 247, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
  File "/usr/lib/python3.8/urllib/request.py", line 563, in error
    result = self._call_chain(*args)
  File "/usr/lib/python3/dist-packages/requests/api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "/usr/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/lib/python3.8/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/usr/lib/python3.8/urllib/request.py", line 1397, in https_open
    return self.do_

  File "/usr/lib/python3.8/http/client.py", line 1418, in connect
    super().connect()
  File "/usr/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.8/http/client.py", line 1425, in connect
    self.sock = self._context.wrap_socket(self.sock,
  File "/usr/lib/python3.8/ssl.py", line 500, in wrap_socket
    return self.sslsocket_class._create(
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 314, in connect
    conn = self._new_conn()
  File "/usr/lib/python3.8/urllib/request.py", line 755, in http_error_302
    return self.parent.open(new, timeout=req.timeout)
  File "/usr/lib/python3.8/ssl.py", line 1040, in _create
    self.do_handshake()
  File "/usr/lib/python3.8/http/client.py", line 1251, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.8/ssl.py", line 500, in wrap_socket
    return self.sslsocket_class._create(
  File "/usr/lib/python3/dist-pac

In [10]:
def show_empty_folder(path)
    dirs = os.listdir(path)
    for d in dirs:
        print(d)
        if os.listdir(f"{path}/{d}") == []:
            print(d)

show_empty_folder('../A')

Archimedes, 280? BC-211? BC
Archimedes, 280? BC-211? BC
Ajax
Aria, Mrs. (Eliza Davis), 1866-1931
Alcock, Deborah, 1835-1913
Allison, Young Ewing, 1853-1932
Avary, Myrta Lockett, 1857-1946
Armstrong, G. H. (George Henry), 1858-1938
Atkinson, Henry A., 1877-1960
Allen, Gladys
Alcott, Louisa May, 1832-1888
Arnold, William B.
Ainslie, Arthur
Archard, Charles J.
Ashley, George T. (George Thomas)
Alexander, Mrs., 1825-1902
Avison, George, 1885-
Abraham, G. P. (George Perry), 1844-1923
Appel, Toby A., 1945-
Archer, Lee
Arnold, Anna E. (Anna Estelle), 1879-1942
Adams, H. A., Mrs.
Alexis, Willibald, 1798-1871
Achilli, Giacinto, 1803-
Armstrong, James
Andersen Nexø, Martin, 1869-1954
Audoux, Marguerite, 1863-1937
Argentina. Ministerio de Agricultura
Anshutz, Edward Pollock, 1846-1918
Alexander, E. A.
Arthur, Chester Alan, 1829-1886
Armstrong, Annie E., 1821-1875
Ammyeetis, 1835-
Andrew, George T.
Anderson, Ellen L.
Arnold, Whately C.
Abercrombie, Lascelles, 1881-1938
Abercrombie, David, 1909-199

In [27]:
links = get_linc_from_dict(pages)

{'https://www.gutenberg.org/ebooks/29666': 'Hymns and Hymnwriters of Denmark'}
{'https://www.gutenberg.org/ebooks/61217': '1,492,633 Marlon Brandos'}
{'https://www.gutenberg.org/ebooks/26348': 'Lisbeth Longfrock'}
{'https://www.gutenberg.org/ebooks/11861': "Radio Boys Cronies; Or, Bill Brown's Radio"}
{'https://www.gutenberg.org/ebooks/25753': 'Radio Boys Loyalty; Or, Bill Brown Listens In'}
{'https://www.gutenberg.org/ebooks/10338': 'With the Turks in Palestine'}
{'https://www.gutenberg.org/ebooks/16791': 'The English Church in the Eighteenth Century'}
{'https://www.gutenberg.org/ebooks/67167': 'Old Songs'}
{'https://www.gutenberg.org/ebooks/62187': '"The Quiet Life": Certain Verses by Various Hands'}
{'https://www.gutenberg.org/ebooks/23037': 'Stories in Verse'}
{'https://www.gutenberg.org/ebooks/17189': 'Autumn Leaves: Original Pieces in Prose and Verse'}
{'https://www.gutenberg.org/ebooks/19768': "Professor Royce's Libel\rA Public Appeal for Redress to the Corporation and Overseers

In [152]:
def get_dictionary(tags):
    name = ''
    result = {}
    for tag in tags:
        if tag.name == 'th':
            name = tag.text
        elif tag.name == 'td' and name != '':
            result = {name: tag.text.rstrip().lstrip()}
    return result

In [ ]:
isAuthor = 'Author' in dictionary
            isTitle = 'Title' in dictionary

In [220]:
def get_list_dictionary(link):
    list_info = []
    page = get_text_html(link)
    data = page.find("table", {"class": "bibrec"}).find_all('tr')
    for tr in data:
        dictionary = get_dictionary(tr)
#         print(dictionary)
        if len(dictionary) > 0:
            res = 'Downloads' in dictionary
            if res == False:
                list_info.append(dictionary)
    return list_info

In [221]:
def delete_duplicate(list_dict):
    keys = []
    for item in list_dict:
        keys.append(list(item.keys())[0])
    unique_keys = set(keys)
    new_dict = {}
    for key in unique_keys:
        new_list = []
        for item in list_dict:
            #print(key, item, key in item)
            if key in item:
                new_list.append(item[key])
        new_dict[key] = new_list
    return new_dict

In [238]:
def get_raw_data(pages):
    print(len(pages))
    list_dictionaty_without_duplicate = []
    count = 0
    for key in pages.keys():
        print(count, key)
        for item in pages[key]:
            list_dictionary = get_list_dictionary(item)
            list_dictionaty_without_duplicate.append(delete_duplicate(list_dictionary))
        count += 1
        if count > 50:
            break
    return list_dictionaty_without_duplicate

In [239]:
raw_data = get_raw_data(pages)

924
0 Aaberg, J. C. (Jens Christian), 1877-1970
1 Aandahl, Vance, 1942-
2 Aanrud, Hans, 1863-1953
3 Aaron, S. F. (Samuel Francis), 1862-
4 Aaronsohn, Alexander, 1888-1948
5 Abbey, Charles J. (Charles John), 1833-1919
6 Abbey, Edwin Austin, 1852-1911
7 Abbey, Henry, 1842-1911
8 Abbot, Anne W. (Anne Wales), 1808-1908
9 Abbot, Francis Ellingwood, 1836-1903
10 Abbot, Robert, cook
11 Abbott, Angus Evan, 1862-1923
12 Abbott, Belle K. (Belle Kendrick), 1842-1893
13 Abbott, Charles C. (Charles Conrad), 1843-1919
14 Abbott, David P. (David Phelps), 1863-1934
15 Abbott, Edwin Abbott, 1838-1926
16 Abbott, Eleanor Hallowell, 1872-1958
17 Abbott, Elenore Plaisted, 1876-1935
18 Abbott, Ernest Hamlin, 1870-1931
19 Abbott, Evelyn, 1843-1901
20 Abbott, Frank Frost, 1860-1924
21 Abbott, Franklin Pierce
22 Abbott, G. F. (George Frederick), 1874-1947
23 Abbott, Henry, 1850-1943
24 Abbott, Jacob, 1803-1879
25 Abbott, Jane, 1881-1968
26 Abbott, John S. C. (John Stevens Cabot), 1805-1877
27 Abbott, Keene, 18

In [244]:
def get_json(dictionary):
    return json.dumps(dictionary, indent = 0)

In [247]:
# for item in raw_data:
#     print(get_json(item))
#     break

In [273]:
def create_csv(name, data):
    header = ['Author', 'Title']
    with open(name, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i in data:
            if 'Author' in i:
                for author in i['Author']:
                    for title in i['Title']:
                        writer.writerow([author, title])
            if 'Contributor' in i:
                for contributor in i['Contributor']:
                    for title in i['Title']:
                        writer.writerow([contributor, title])

create_csv('parsed_data.csv', raw_data)

In [7]:
def get_list_link(page):
    links = []
    pgdbbyauthor = page.find("div", {"class": "pgdbbyauthor"})
    for element in pgdbbyauthor.children:
        if element.name == 'h2':
            pass
        if element.name == 'ul':
            for link in element.find_all('a'):
                link_key = link['href']
                if 'ebooks' in link_key:
                    links.append({'https://www.gutenberg.org' + link_key: link.text})
    return links

In [10]:
links

[]

In [11]:
def create_file(text, path):
    if os.path.isfile(path) == False:
        with open(path + '.txt', 'w') as file: 
            file.write(text) 

In [13]:
def get_link_for_download(url):
    content = get_content(url)
    elements = BeautifulSoup(content, 'html.parser').find('table').find_all('tr')
    for element in elements:
        for attribute in element.attrs:
            if attribute == 'about':
                if '.txt' in element['about']:
                    return element['about']

In [14]:
def create_file_by_linc(link, dir_name):
    key = list(link.keys())[0]
    keu_for_download = get_link_for_download(key)
    r = requests.get(keu_for_download, stream=True)
    if r.status_code == 200:
        with open(dir_name + '/' + link[key], 'wb') as f:
            for chunk in r:
                f.write(chunk)
    #urllib.request.urlretrieve(keu_for_download, dir_name + '/' + link[key])

In [15]:
def download_files(links):
    dir_name = 'A'
    create_dir(dir_name)
    for link in links:
        create_file_by_linc(link, dir_name)
    #ThreadPool(9).imap_unordered(create_file_by_linc, links)

In [16]:
download_files(links)